## Data Loading

Merging the data from PJM (https://dataminer2.pjm.com/feed/hrl_load_metered) downloaded for each of the year and saving them into one CSV file which will be used in the main Notebook.

In [6]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [5]:

files = ['hrl_load_metered_CE_2015.csv', 'hrl_load_metered_CE_2016.csv', 'hrl_load_metered_CE_2017.csv', 'hrl_load_metered_CE_2018.csv',
        'hrl_load_metered_CE_2019.csv', 'hrl_load_metered_CE_2020.csv', 'hrl_load_metered_CE_2021.csv', 'hrl_load_metered_CE_2022.csv',
        'hrl_load_metered_CE_2023.csv', 'hrl_load_metered_CE_2024.csv']

result = []

for file in files:
    read_csv = pd.read_csv(file)
    result.append(read_csv)

df = pd.concat(result, ignore_index = True)
df.head()
df.to_csv('hrl_load_metered_CE_2015_2024.csv', index = False)
print('Finished')

Finished


Get hourly weather data for Chicago from NASA  
Time = LT (Local Time)

In [26]:
# Define the coordinates for the location (Chicago)
latitude = 41.8818   
longitude = -87.6232 

# Define the start and end dates in YYYYMMDD format
start_date = "20141231"  # Start date (YYYYMMDD)
end_date = "20241231"  # End date (YYYYMMDD)

# Define the weather parameters to fetch
parameters = "T2M"  
# T2M - Temperature at 2 meters  

# Construct the API request URL
api_url = f"https://power.larc.nasa.gov/api/temporal/hourly/point?Time=LT&parameters={parameters}&community=RE&longitude={longitude}&latitude={latitude}&start={start_date}&end={end_date}&format=CSV"

# Send the request to the NASA POWER API
response = requests.get(api_url)

# Check if the request was successful
if response.status_code == 200:
    # Save the response content to a CSV file
    with open("weather_data.csv", "wb") as file:
        file.write(response.content)  
    print("File downloaded successfully: weather_data.csv")
else:
    print("Failed to download the data.")

File downloaded successfully: weather_data.csv


Reading of downloaded file and creation of the dataframe

In [27]:

df = pd.read_csv('weather_data.csv', delimiter = ',', skiprows = 9)
df = df.iloc[14:]
df.reset_index(drop = True, inplace = True)

In [28]:
df.dtypes

YEAR      int64
MO        int64
DY        int64
HR        int64
T2M     float64
dtype: object

In [29]:
df.head(20)

,YEAR,MO,DY,HR,T2M
0,2014,12,31,14,-6.30
1,2014,12,31,15,-5.95
2,2014,12,31,16,-5.84
3,2014,12,31,17,-5.71
4,2014,12,31,18,-5.57
5,2014,12,31,19,-5.52
6,2014,12,31,20,-5.53
7,2014,12,31,21,-5.59
8,2014,12,31,22,-5.72
9,2014,12,31,23,-5.88


In [30]:
# delete 10 hours corresponding to 2014
df = df.iloc[10:].reset_index(drop = True)

In [31]:
df.head()

,YEAR,MO,DY,HR,T2M
0,2015,1,1,0,-6.02
1,2015,1,1,1,-6.17
2,2015,1,1,2,-6.37
3,2015,1,1,3,-6.55
4,2015,1,1,4,-6.72


In [32]:
df.nunique()

YEAR      10
MO        12
DY        31
HR        24
T2M     4625
dtype: int64

In [34]:
# create column datetime

df['datetime_beginning_ept'] = pd.to_datetime(df[['YEAR', 'MO', 'DY', 'HR']]
                                .astype(str)
                                .agg('-'.join, axis=1), 
                                format='%Y-%m-%d-%H')
df['datetime_beginning_ept'] = pd.to_datetime(df['datetime_beginning_ept'], format='%m/%d/%Y %I:%M:%S %p')
df.head()

,YEAR,MO,DY,HR,T2M,datetime_beginning_ept
0,2015,1,1,0,-6.02,2015-01-01 00:00:00
1,2015,1,1,1,-6.17,2015-01-01 01:00:00
2,2015,1,1,2,-6.37,2015-01-01 02:00:00
3,2015,1,1,3,-6.55,2015-01-01 03:00:00
4,2015,1,1,4,-6.72,2015-01-01 04:00:00


In [35]:
df.drop(columns=['YEAR', 'MO', 'DY', 'HR'], inplace=True)
df.head()

,T2M,datetime_beginning_ept
0,-6.02,2015-01-01 00:00:00
1,-6.17,2015-01-01 01:00:00
2,-6.37,2015-01-01 02:00:00
3,-6.55,2015-01-01 03:00:00
4,-6.72,2015-01-01 04:00:00


In [36]:
df.to_csv('weather_data_preprocessed.csv', index = False)